# Regional economic dynamics, nighttime lights, and land cover: Insights from dimly lit islands

## Install Packages

In [84]:
import pandas as pd
# import geopandas as gpd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from pandasgui import show

## Preliminaries

In [85]:
# Set path
path = "/Users/Jesson Pagaduan/Google Drive/Geospatial_Project/"

In [86]:
df_ntl_prov = pd.read_csv(path + "data/scratch/NTL_2000-2018_PHL_province_21Apr.csv", index_col=False,
                          usecols=['YEAR', 'NAME_1', 'ZONE_CODE', 'COUNT', 'SUM'], 
                          parse_dates=['YEAR'])
df_ntl_prov.columns = [i.lower() for i in df_ntl_prov.columns]

In [87]:
df_ntl_prov['region'] = ''

In [88]:
ncr = ['Metropolitan Manila']
car = ['Abra', 'Apayao', 'Benguet', 'Ifugao', 'Kalinga', 'Mountain Province']
region1 = ['Ilocos Norte', 'Ilocos Sur', 'La Union', 'Pangasinan']
region2 = ['Batanes', 'Cagayan', 'Isabela', 'Nueva Vizcaya', 'Quirino']
region3 = ['Aurora', 'Bataan', 'Bulacan', 'Nueva Ecija', 'Pampanga', 'Tarlac', 'Zambales']
region4a = ['Batangas', 'Cavite', 'Laguna', 'Quezon', 'Rizal']
region4b = ['Marinduque', 'Occidental Mindoro', 'Oriental Mindoro', 'Palawan', 'Romblon']
region5 = ['Albay', 'Camarines Norte', 'Camarines Sur', 'Catanduanes', 'Masbate', 'Sorsogon']
region6 = ['Aklan', 'Antique', 'Capiz', 'Guimaras', 'Iloilo', 'Negros Occidental']
region7 = ['Bohol', 'Cebu', 'Negros Oriental', 'Siquijor']
region8 = ['Biliran', 'Eastern Samar', 'Leyte', 'Northern Samar', 'Samar', 'Southern Leyte']
region9 = ['Zamboanga del Norte', 'Zamboanga del Sur', 'Zamboanga Sibugay']
region10 = ['Bukidnon', 'Camiguin', 'Lanao del Norte', 'Misamis Occidental', 'Misamis Oriental']
region11 = ['Compostela Valley', 'Davao del Norte', 'Davao del Sur', 'Davao Oriental']
region12 = ['North Cotabato', 'Sarangani', 'South Cotabato', 'Sultan Kudarat']
region13 = ['Agusan del Norte', 'Agusan del Sur', 'Dinagat Islands', 'Surigao del Norte', 'Surigao del Sur']
armm = ['Basilan', 'Lanao del Sur', 'Maguindanao', 'Sulu', 'Tawi-tawi']

In [89]:
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(ncr), 'region'] = 'Metropolitan Manila'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(car), 'region'] = 'Cordillera Administrative Region (CAR)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region1), 'region'] = 'Ilocos Region (Region I)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region2), 'region'] = 'Cagayan Valley (Region II)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region3), 'region'] = 'Central Luzon (Region III)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region4a), 'region'] = 'CALABARZON (Region IV-A)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region4b), 'region'] = 'MIMAROPA (Region IV-B)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region5), 'region'] = 'Bicol Region (Region V)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region6), 'region'] = 'Western Visayas (Region VI)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region7), 'region'] = 'Central Visayas (Region VII)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region8), 'region'] = 'Eastern Visayas (Region VIII)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region9), 'region'] = 'Zamboanga Peninsula (Region IX)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region10), 'region'] = 'Northern Mindanao (Region X)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region11), 'region'] = 'Davao Region (Region XI)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region12), 'region'] = 'SOCCSKSARGEN (Region XII)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region13), 'region'] = 'Caraga (Region XIII)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(armm), 'region'] = 'Autonomous Region of Muslim Mindanao (ARMM)'

In [90]:
df_ntl_prov = df_ntl_prov[['year', 'region', 'name_1', 'zone_code', 'count', 'sum']]

In [91]:
df_ntl_prov

,year,region,name_1,zone_code,count,sum
0,2018-01-01,Cordillera Administrative Region (CAR),Abra,1,16754,57.640000
1,2018-01-01,Caraga (Region XIII),Agusan del Norte,2,12207,1434.179999
2,2018-01-01,Caraga (Region XIII),Agusan del Sur,3,34906,373.640000
3,2018-01-01,Western Visayas (Region VI),Aklan,4,6844,459.970000
4,2018-01-01,Bicol Region (Region V),Albay,5,10395,1369.459998
...,...,...,...,...,...,...
1534,2000-01-01,,Tawi-Tawi,77,4754,0.000000
1535,2000-01-01,Central Luzon (Region III),Zambales,78,15246,721.129173
1536,2000-01-01,Zamboanga Peninsula (Region IX),Zamboanga del Norte,79,25971,12.207475
1537,2000-01-01,Zamboanga Peninsula (Region IX),Zamboanga del Sur,80,21520,1052.154341


In [92]:
df_ntl_reg = pd.read_csv(path + "data/scratch/data.csv", index_col=False,
                         usecols=['YEAR', 'REGION', 'ZONE_CODE', 'NTL_COUNT', 'NTL_SUM'], 
                         parse_dates=['YEAR'])
df_ntl_reg.columns = [i.lower() for i in df_ntl_reg.columns]

In [93]:
df_ntl_reg

,year,region,zone_code,ntl_count,ntl_sum
0,2000-01-01,Autonomous Region of Muslim Mindanao (ARMM),1,51776,401.994348
1,2001-01-01,Autonomous Region of Muslim Mindanao (ARMM),1,51776,563.428316
2,2002-01-01,Autonomous Region of Muslim Mindanao (ARMM),1,51776,555.334398
3,2003-01-01,Autonomous Region of Muslim Mindanao (ARMM),1,51776,569.406300
4,2004-01-01,Autonomous Region of Muslim Mindanao (ARMM),1,51776,503.546102
...,...,...,...,...,...
318,2014-01-01,Zamboanga Peninsula (Region IX),17,58494,2707.028250
319,2015-01-01,Zamboanga Peninsula (Region IX),17,58494,2724.677521
320,2016-01-01,Zamboanga Peninsula (Region IX),17,58494,2288.837433
321,2017-01-01,Zamboanga Peninsula (Region IX),17,58494,2799.646658


In [94]:
df_ntl_prov = df_ntl_reg.set_index(['region', 'year']).join(df_ntl_prov.set_index(['region', 'year']),
                                                            rsuffix='_prov')

In [95]:
df_ntl_prov.columns = ['reg_id', 'reg_count_lit', 'reg_ntl', 'province', 'prov_id', 'prov_count_lit',
                       'prov_ntl']

In [96]:
df_ntl_prov = df_ntl_prov.assign(prov_ntl_share = lambda x: x.prov_ntl / x.reg_ntl)

In [97]:
df_ntl_prov

reg_id  reg_count_lit  \
region                                      year                                
Autonomous Region of Muslim Mindanao (ARMM) 2000-01-01       1          51776   
                                            2000-01-01       1          51776   
                                            2000-01-01       1          51776   
                                            2000-01-01       1          51776   
                                            2001-01-01       1          51776   
...                                                        ...            ...   
Zamboanga Peninsula (Region IX)             2017-01-01      17          58494   
                                            2017-01-01      17          58494   
                                            2018-01-01      17          58494   
                                            2018-01-01      17          58494   
                                            2018-01-01      17          58494   

                                                            reg_ntl  \
region                                      year                      
Autonomous Region of Muslim Mindanao (ARMM) 2000-01-01   401.994348   
                                            2000-01-01   401.994348   
                                            2000-01-01   401.994348   
                                            2000-01-01   401.994348   
                                            2001-01-01   563.428316   
...                                                             ...   
Zamboanga Peninsula (Region IX)             2017-01-01  2799.646658   
                                            2017-01-01  2799.646658   
                                            2018-01-01  3069.750007   
                                            2018-01-01  3069.750007   
                                            2018-01-01  3069.750007   

                                                                   province  \
region                                      year                              
Autonomous Region of Muslim Mindanao (ARMM) 2000-01-01              Basilan   
                                            2000-01-01        Lanao del Sur   
                                            2000-01-01          Maguindanao   
                                            2000-01-01                 Sulu   
                                            2001-01-01              Basilan   
...                                                                     ...   
Zamboanga Peninsula (Region IX)             2017-01-01    Zamboanga del Sur   
                                            2017-01-01    Zamboanga Sibugay   
                                            2018-01-01  Zamboanga del Norte   
                                            2018-01-01    Zamboanga del Sur   
                                            2018-01-01    Zamboanga Sibugay   

                                                        prov_id  \
region                                      year                  
Autonomous Region of Muslim Mindanao (ARMM) 2000-01-01        9   
                                            2000-01-01       42   
                                            2000-01-01       44   
                                            2000-01-01       73   
                                            2001-01-01        9   
...                                                         ...   
Zamboanga Peninsula (Region IX)             2017-01-01       80   
                                            2017-01-01       81   
                                            2018-01-01       79   
                                            2018-01-01       80   
                                            2018-01-01       81   

                                                        prov_count_lit  \
region                                      year                         
Autonomous Region of Muslim Mi

In [98]:
df_predictions = pd.read_excel(path + "gdp_predictions.xlsx", index_col=False,
                               usecols=['region', 'year', 'period', 'zone_code', 'ind_ntl_top13_p1', 
                                        'svc_ntl_top13_p1', 'non_agr_ntl_top13_p1', 'ind_ntl_top13_p2',
                                        'svc_ntl_top13_p2', 'non_agr_ntl_top13_p2'], 
                               parse_dates=['year'])
df_predictions.columns = [i.lower() for i in df_predictions.columns]

In [99]:
df_ntl_prov = df_ntl_prov.join(df_predictions.set_index(['region', 'year']), rsuffix='_pred')

In [101]:
df_ntl_prov.assign(log_ind_p1 = lambda x: x.ind_ntl_top13_p1 * x.prov_ntl_share,
                   log_ind_p2 = lambda x: x.ind_ntl_top13_p2 * x.prov_ntl_share,
                   log_svc_p1 = lambda x: x.svc_ntl_top13_p1 * x.prov_ntl_share,
                   log_svc_p2 = lambda x: x.svc_ntl_top13_p2 * x.prov_ntl_share,
                   log_non_agr_p1 = lambda x: x.non_agr_ntl_top13_p1 * x.prov_ntl_share,
                   log_non_agr_p2 = lambda x: x.non_agr_ntl_top13_p2 * x.prov_ntl_share)

reg_id  reg_count_lit  \
region                                      year                                
Autonomous Region of Muslim Mindanao (ARMM) 2000-01-01       1          51776   
                                            2000-01-01       1          51776   
                                            2000-01-01       1          51776   
                                            2000-01-01       1          51776   
                                            2001-01-01       1          51776   
...                                                        ...            ...   
Zamboanga Peninsula (Region IX)             2017-01-01      17          58494   
                                            2017-01-01      17          58494   
                                            2018-01-01      17          58494   
                                            2018-01-01      17          58494   
                                            2018-01-01      17          58494   

                                                            reg_ntl  \
region                                      year                      
Autonomous Region of Muslim Mindanao (ARMM) 2000-01-01   401.994348   
                                            2000-01-01   401.994348   
                                            2000-01-01   401.994348   
                                            2000-01-01   401.994348   
                                            2001-01-01   563.428316   
...                                                             ...   
Zamboanga Peninsula (Region IX)             2017-01-01  2799.646658   
                                            2017-01-01  2799.646658   
                                            2018-01-01  3069.750007   
                                            2018-01-01  3069.750007   
                                            2018-01-01  3069.750007   

                                                                   province  \
region                                      year                              
Autonomous Region of Muslim Mindanao (ARMM) 2000-01-01              Basilan   
                                            2000-01-01        Lanao del Sur   
                                            2000-01-01          Maguindanao   
                                            2000-01-01                 Sulu   
                                            2001-01-01              Basilan   
...                                                                     ...   
Zamboanga Peninsula (Region IX)             2017-01-01    Zamboanga del Sur   
                                            2017-01-01    Zamboanga Sibugay   
                                            2018-01-01  Zamboanga del Norte   
                                            2018-01-01    Zamboanga del Sur   
                                            2018-01-01    Zamboanga Sibugay   

                                                        prov_id  \
region                                      year                  
Autonomous Region of Muslim Mindanao (ARMM) 2000-01-01        9   
                                            2000-01-01       42   
                                            2000-01-01       44   
                                            2000-01-01       73   
                                            2001-01-01        9   
...                                                         ...   
Zamboanga Peninsula (Region IX)             2017-01-01       80   
                                            2017-01-01       81   
                                            2018-01-01       79   
                                            2018-01-01       80   
                                            2018-01-01       81   

                                                        prov_count_lit  \
region                                      year                         
Autonomous Region of Muslim Mi

In [83]:
df_ntl_prov

reg_id  reg_count_lit  \
region                                      year                                
Autonomous Region of Muslim Mindanao (ARMM) 2000-01-01       1          51776   
                                            2000-01-01       1          51776   
                                            2000-01-01       1          51776   
                                            2000-01-01       1          51776   
                                            2001-01-01       1          51776   
...                                                        ...            ...   
Zamboanga Peninsula (Region IX)             2017-01-01      17          58494   
                                            2017-01-01      17          58494   
                                            2018-01-01      17          58494   
                                            2018-01-01      17          58494   
                                            2018-01-01      17          58494   

                                                            reg_ntl  \
region                                      year                      
Autonomous Region of Muslim Mindanao (ARMM) 2000-01-01   401.994348   
                                            2000-01-01   401.994348   
                                            2000-01-01   401.994348   
                                            2000-01-01   401.994348   
                                            2001-01-01   563.428316   
...                                                             ...   
Zamboanga Peninsula (Region IX)             2017-01-01  2799.646658   
                                            2017-01-01  2799.646658   
                                            2018-01-01  3069.750007   
                                            2018-01-01  3069.750007   
                                            2018-01-01  3069.750007   

                                                                   province  \
region                                      year                              
Autonomous Region of Muslim Mindanao (ARMM) 2000-01-01              Basilan   
                                            2000-01-01        Lanao del Sur   
                                            2000-01-01          Maguindanao   
                                            2000-01-01                 Sulu   
                                            2001-01-01              Basilan   
...                                                                     ...   
Zamboanga Peninsula (Region IX)             2017-01-01    Zamboanga del Sur   
                                            2017-01-01    Zamboanga Sibugay   
                                            2018-01-01  Zamboanga del Norte   
                                            2018-01-01    Zamboanga del Sur   
                                            2018-01-01    Zamboanga Sibugay   

                                                        prov_id  \
region                                      year                  
Autonomous Region of Muslim Mindanao (ARMM) 2000-01-01        9   
                                            2000-01-01       42   
                                            2000-01-01       44   
                                            2000-01-01       73   
                                            2001-01-01        9   
...                                                         ...   
Zamboanga Peninsula (Region IX)             2017-01-01       80   
                                            2017-01-01       81   
                                            2018-01-01       79   
                                            2018-01-01       80   
                                            2018-01-01       81   

                                                        prov_count_lit  \
region                                      year                         
Autonomous Region of Muslim Mi